# RERANK
- 트랜스포머(버트) 크로스인코더 구조 활용해서 더욱 정밀한 시맨틱 유사도 스코어링 및 이에 기반한 문서 재정렬

In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(model='gpt-4o-mini')

In [4]:
# load documents
documents = SimpleDirectoryReader("./content/pg").load_data()

# build index
index = VectorStoreIndex.from_documents(documents=documents)

In [2]:
import os
from llama_index.postprocessor.cohere_rerank import CohereRerank

#cohere api 키
api_key = ''

#리랭커 top_2로 정의
cohere_rerank = CohereRerank(api_key=api_key, top_n=2)

In [5]:
# 어차피 리랭크로 재소팅할거니까 넉넉한 top_k값 부여
query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[cohere_rerank],
)
response = query_engine.query(
    "What did Sam Altman do in this essay?",
)

In [6]:
pprint_response(response, show_source=True)

Final Response: Sam Altman was approached to become the president of Y
Combinator (YC) after the author realized he was ready to hand over
the responsibilities of YC to someone else. Initially, Altman declined
the offer as he wanted to start a startup focused on making nuclear
reactors. However, after some persistence, he agreed to take over
starting with the winter 2014 batch. The transition involved a
complete changing of the guard at YC, with the original founders
stepping back to allow for new leadership.
______________________________________________________________________
Source Node 1/2
Node ID: 9adad3b1-e678-4c3e-b782-0a7bbac54e6c
Similarity: 0.83999664
Text: One day in 2010, when he was visiting California for interviews,
Robert Morris did something astonishing: he offered me unsolicited
advice. I can only remember him doing that once before. One day at
Viaweb, when I was bent over double from a kidney stone, he suggested
that it would be a good idea for him to take me to the

In [7]:
# naive 쿼리 결과 비교
query_engine = index.as_query_engine(
    similarity_top_k=2,
)
response = query_engine.query(
    "What did Sam Altman do in this essay?",
)

In [8]:
pprint_response(response, show_source=True)

Final Response: Sam Altman is not mentioned in the provided text, so
there is no information about his actions or contributions in this
essay. The focus is primarily on the author's experiences and thoughts
regarding writing essays, the evolution of publishing, and his
ventures in technology and startups.
______________________________________________________________________
Source Node 1/2
Node ID: 03f712ca-75b7-43a9-aed0-06a2bd37bef4
Similarity: 0.5322797769741822
Text: Except for a few officially anointed thinkers who went to the
right parties in New York, the only people allowed to publish essays
were specialists writing about their specialties. There were so many
essays that had never been written, because there had been no way to
publish them. Now they could be, and I was going to write them. [12]
I've wor...
______________________________________________________________________
Source Node 2/2
Node ID: 6e94e3f2-8179-4272-8766-6c04bbb318c4
Similarity: 0.5234146341021664
Text: I r

In [9]:
# Dataset 로드
from datasets import load_dataset

ds = load_dataset("HAERAE-HUB/KOREAN-WEBTEXT", split='train[:20]')
data = ds.to_pandas()

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [10]:
# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
docs = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        # extra_info={'title': row['title']}
    ))

In [11]:
index = VectorStoreIndex.from_documents(
    docs
)

In [12]:
# naive engine
query_engine = index.as_query_engine(
    similarity_top_k=2
)

# rerank engine
rerank_query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[cohere_rerank],
)

In [14]:
# naive Response
naive_response = query_engine.query(
    "분당과 같은 도시가 외국에서도 통하려면 어떻게해야되?",
)
#rerank response
rerank_response = rerank_query_engine.query(
    "분당과 같은 도시가 외국에서도 통하려면 어떻게해야되?",
)

In [15]:
pprint_response(naive_response, show_source=True)
print('------------------------------RERANK-----------------------------')
pprint_response(rerank_response, show_source=True)

Final Response: 외국에서 분당과 같은 도시가 성공적으로 통하려면, 해당 지역의 사람들, 문화, 시장, 관계, 제도
등을 깊이 이해하는 것이 중요합니다. 또한, 시행과 시공이 분리되어야 하며, 거시적 및 미시적 경제상황에 따라 주택시장에
유연하게 대응할 수 있는 주택상품을 기획해야 합니다. 로컬의 특성을 잘 반영하고, 현지의 요구에 맞춘 접근이 필요합니다.
______________________________________________________________________
Source Node 1/2
Node ID: bb42483e-9d2d-4da0-ab94-ca60731bfae8
Similarity: 0.27837277228282153
Text: 이러한 비정기적인 지출에 대비해 #2.예비비통장을 만들어 두어야 합니다. 이 예비비통장에는 매월 정기 수입의 10%
정도를 떼어서 입금해 두는 것이 좋습니다. 이 통장에 돈이 없으면 신용카드나 마이너스 대출, 카드론 등을 사용하게 되는데 이는
지출의 악순환이 반복되는 지름길입니다. 정기적인 지출은 재테크 통장과 생활비 통장으로 구분할 수 있습니다. #3. 재테크
통장은 매월 고정적으로 나가는 저축・투자 및 지출 항목을 자동이체로 설정해 두는 통장입니다. 재테크통장에서 이체되는 돈은
적금, 보험, 연금, 펀드, 대출원리금 등입니다. 이때 월급을 나누는 비율이 중요합니다. 적금, 펀드 등을 위한
저축・투자통장으로 15...
______________________________________________________________________
Source Node 2/2
Node ID: 91f082a0-e93d-4f61-8355-c5a6fba121de
Similarity: 0.259410312844732
Text: 초기 분양상품은 한채 약 7만불 내외의 흥붕 및 스카이가든 아파트였고 분양은 매우 성공적이었다. 많은 사람들이
베트남에서 아파트문화를 새롭게 정의하는데 주도적 역활을 했다. 이후에 중

In [16]:
# naive Response
naive_response = query_engine.query(
    "프랭키가 누구한테 인기가 없을까?",
)
#rerank response
rerank_response = rerank_query_engine.query(
    "프랭키가 누구한테 인기가 없을까?",
)

In [17]:
pprint_response(naive_response, show_source=True)
print('------------------------------RERANK-----------------------------')
pprint_response(rerank_response, show_source=True)

Final Response: 프랭키는 일본 축구에 대한 한국인의 인식 속에서 인기가 없을 것으로 보인다. 한국에서는 일본
축구에 대해 사실과 다른 객기를 부리는 경향이 있으며, 이는 프랭키와 같은 인물에게 부정적인 영향을 미칠 수 있다.
______________________________________________________________________
Source Node 1/2
Node ID: ee97662e-2585-4f9d-aa6f-f73c9ae04e47
Similarity: 0.21386543223725202
Text: 일기라는 가장 내밀한 기록 속에 드러나는 세계제국 영국 역사의 치부, 성이라는 가장 사적인 생활에 개입된 계급적
한계가 사회 최상류층이자 최약자로서 찰스와 윌을 통해 구현되면서 소설은 두터운 현실감을 얻는다. 작가 특유의 정교한 구성과
섬세하고 날렵한 문체에 읽는 재미를 더하는 것은 군데군데 숨은 풍자다. 영국 왕위계승 서열을 그대로 따른 주요 인물들의 이름,
저소득층을 상대로 자선을 베푸는 한편 그들을 꾀어 포르노 영화를 찍으며 감상하는 사진가의 이름에 오점, 얼룩(stain)의
뜻이 담긴 점, 아르헨띠나인과 영국의 포클랜드 침략에 관해 대화를 나누면서 윌이 거꾸로 희롱당하는 입장에 처하는 장면 등은
우스꽝스럽게...
______________________________________________________________________
Source Node 2/2
Node ID: cb4b533e-0add-401c-9c21-1d4c4a18d173
Similarity: 0.20551013866250534
Text: 아마도 대한민국에서 일본보다 우리나라 성인축구 수준이 한참이나 떨어진다고 하면 이를 인정하는 사람은 거의 없다.
언제나 일본축구보다 우위에 있다는 정신승리자위질에 빠져 있다. 아시안컵에서 우리나라는 50년대 2번 우승한 것을 제외하고는
지금까지 단 한번도 우승한 적이 없다. 그러나 일본은 92년부터 총 8번의 

In [18]:
# naive Response
naive_response = query_engine.query(
    "동양 섬나라 국가랑 서양국가가 싸운 전쟁이 뭐였지?",
)
#rerank response
rerank_response = rerank_query_engine.query(
    "동양 섬나라 국가랑 서양국가가 싸운 전쟁이 뭐였지?",
)

In [19]:
pprint_response(naive_response, show_source=True)
print('------------------------------RERANK-----------------------------')
pprint_response(rerank_response, show_source=True)

Final Response: 동양 섬나라 국가와 서양 국가 간의 전쟁에 대한 정보는 제공된 내용에 포함되어 있지 않습니다.
______________________________________________________________________
Source Node 1/2
Node ID: 0118dc93-d1f0-4c38-ace7-41cca0af1394
Similarity: 0.30642135093822115
Text: “아~아~잊으랴 어찌 우리 이날을 조국의 원수들이 짓밟아 오던 날을~”6·25의 노래 시작부이다. 1950년에
발발하여 남북한 인구 600만명 이상의 인명피해를 남기고 전국토의 80%이상을 파괴하였던 6·25 전쟁은 발발 3년 뒤인
1953년 7월에 휴전이 되어 지금에까지 이르고있다.1955년 6월은 6·25에 대한 기억이 생생할 때이다. 1955년 6월
25일에는 6·25 발발 다섯돌을 맞아 국민대회가 폭우를 무릎쓰고 진행되었다고 당시 일간지들은 보도하고 있다. 하나님께서는
1955년 6월 9일부터 16일까지 대구집회를 마치고 일주일 뒤인 6월 23일부터 30일까지 용산 군인교회에서 집회를
가지셨다. 신문에 나간...
______________________________________________________________________
Source Node 2/2
Node ID: 621b3057-f3cb-4314-a10d-58da5660e67a
Similarity: 0.2843911447654196
Text: 교묘하게 반일감정을 자극하여 조센징 개돼지들을 선동하고 있다. 일본의 경제력은 한국과 프랑스의 경제력을 합쳐 놓은
것과 비슷하다. 한국의 경제력은 정확하게 프랑스 경제력의 절반이다. 앞으로 일본 경제력을 추월할 일은 벌어지지 않을 것이고,
남북통일이 되어도 일본의 경제력을 능가할 가능성은 없다. 게다가, 국민성 마져도 일본은 적에게서 배우고 실패에서 배운다.
이성적이고 공공의 이익을 추구할 줄 안다. 그에 비하여 한국은 이성

In [20]:
# naive Response
naive_response = query_engine.query(
    "빚부터 갚으라는 유튜버가 누구지?",
)
#rerank response
rerank_response = rerank_query_engine.query(
    "빚부터 갚으라는 유튜버가 누구지?",
)

In [21]:
pprint_response(naive_response, show_source=True)
print('------------------------------RERANK-----------------------------')
pprint_response(rerank_response, show_source=True)

Final Response: 오상열입니다.
______________________________________________________________________
Source Node 1/2
Node ID: 1e71d835-c212-4b4f-92d8-779a2c85ea12
Similarity: 0.2703491596571366
Text: 거리와 색상을 통해 자신이 해야 할 일을 판단하는 거죠. 간단하게 1~2 단계 움직이는 모습을 영상으로 담아
봤는데요. 고양이 울음소리를 내고 감정을 표현하는 게 영락없는 고양이입니다. 맨날 싸우는 우리 아이들이지만 레고 부스트를
만드는 시간만큼은 세상 둘도 없는 오누이입니다. 둘째는 자기가 만들지 못할 것을 아니까 구경하면서 응원하고 첫째가 열심히
만들죠. 지난번에도 그러더니 꼭 친한 척을 하네요. 짠 ~ 다리까지 모두 완성된 프랭키입니다. 귀엽죠? 나름 리본도 매달고
있습니다. 울 따님은 너무 귀엽다며 계속 이방 저방 데리고 다니고 있습니다. 기다란 꼬리를 가지고 있는데, 꼬리까지 살랑살랑
흔들면서 진짜 고양이...
______________________________________________________________________
Source Node 2/2
Node ID: c149105b-2278-4078-8982-db4dc6668341
Similarity: 0.2673633113263073
Text: 재테크 채널 유튜버이자, 「빚부터 갚아라」, 「원트재무설계 소원을 말해봐」 저자인 오상열의 <어차피 재테크는 토스>
칼럼 시리즈에서는, 재테크의 가장 기본인 통장 관리부터 채권 투자까지 전반적인 돈 관리 방법을 하나씩 짚게 됩니다. 모두가
경제적 자유를 얻을 수 있도록, 알찬 재테크 정보를 쉽고 재미있게 전달합니다. 월급이 들어오는 통장 관리, 어떻게 하고
계시나요? 월급이란 근로와 사업의 댓가로 매월 통장으로 들어오는 수입을 말합니다. 이 수입을 가지고 생활비로도 쓰고, 저축과
투자도 하고, 보험료도 납